In [1]:
import dataset
data=dataset.UNDevGoalsDataset()
data.prediction_error(error_fn='RMSE', model_name='status_quo_model',pp_fn='preprocess_avg_NANs')

/Users/aleckirkley/anaconda/envs/py3alec/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


0.02389522670468707

In [2]:
X1,Y1=data.preprocess(pp_fn='preprocess_simple')
X2,Y2=data.preprocess(pp_fn='preprocess_avg_NANs')

In [ ]:
data.predictions(model_name='status_quo_model', model_kwargs={},\
                                       pp_fn='preprocess_avg_NANs', pp_fn_kwargs={})

In [ ]:
from collections import Counter
Counter(X1.iloc[:,-5].isnull())

In [ ]:
def VAR_model(X):
    """Predict the most recent value using a VAR model.

    If any issues fitting time series model, use status_quo."""
    
    sq_preds = status_quo_model(X)
    forecasts = list()
    # lots of warnings about convergence. can ignore for now.
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        for index, row in X.iterrows():
            # fill in gaps
            row_interp = row.interpolate(
                method = 'linear', limit = 50,
                limit_direction = 'backward')
            # Fit ARIMA model on `lookback` most recent years data.
            # Since so much missing data exists, it is not
            # clear that including more years of interpolated
            # data is helping in terms of RMSE
            model = sm.tsa.api.VAR(row_interp.tolist()[-lookback:], order=order)
            try:
                results = model.fit(disp = 0)
                if pd.isnull(results.forecast()[0][0]) or np.abs(results.forecast()[0][0])>2:
                    forecasts.append(sq_preds.loc[index])
                else: 
                    forecasts.append(results.forecast()[0][0])
            except (ValueError, np.linalg.linalg.LinAlgError, MissingDataError) as e:
                    forecasts.append(sq_preds.loc[index])
    return(forecasts)

In [33]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.api import VAR, DynamicVAR

X2_stat=X2.diff(axis='columns').iloc[:,1:]
model = VAR(X2_stat.values)
results = model.fit(2)

In [46]:
results.forecast(np.array(Y2).reshape(-1,1),steps=1)

ValueError: shapes (34,34) and (1,) not aligned: 34 (dim 1) != 1 (dim 0)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.array(X2_norm.iloc[1]))

In [23]:
np.array(Y2).reshape(-1,1)

array([[4.65000000e-01],
       [1.90000000e-02],
       [1.11500000e-01],
       [1.00000000e-03],
       [9.61000000e-01],
       [1.50361154e-01],
       [2.03000000e-02],
       [9.39002900e-01],
       [8.44000000e-01],
       [9.45119063e-02],
       [2.41000000e-02],
       [7.08700000e-01],
       [3.80000000e-03],
       [5.02000000e-01],
       [3.20000000e-02],
       [1.87100000e-01],
       [2.10000000e-02],
       [9.18214600e-01],
       [9.79000000e-01],
       [3.40000000e-01],
       [1.18000000e-02],
       [9.79000000e-01],
       [2.59466329e-01],
       [1.59000000e-02],
       [4.00000000e-03],
       [9.67000000e-01],
       [6.02125340e-02],
       [2.11000000e-02],
       [1.00000000e-03],
       [9.65000000e-01],
       [3.09000000e-01],
       [9.62613100e-01],
       [6.94500000e-01],
       [5.30000000e-03],
       [6.93700000e-01],
       [4.40000000e-03],
       [8.34584100e-01],
       [7.93000000e-01],
       [1.45400000e-01],
       [4.53000000e-02],
